In [88]:
import redis

redis_client = redis.Redis(host='localhost', port=6379, db=0)
print("Redis client initialized")
data = redis_client.lrange(name='executed_orders', start=0, end=-1)
data

Redis client initialized


[b'{"order_id": "1764751038790", "symbol": "NSE:BANKNIFTY25DEC59600PE", "price": 745.0, "qty": 100, "status": "BUY", "timestamp": 1764751038.7909732}',
 b'{"order_id": "1764751040906", "symbol": "NSE:BANKNIFTY25DEC59600PE", "price": 743.9, "qty": 100, "status": "SELL", "timestamp": 1764751040.9066985}',
 b'{"order_id": "1764751047966", "symbol": "NSE:BANKNIFTY25DEC59600CE", "price": 612.0, "qty": 100, "status": "BUY", "timestamp": 1764751047.9666693}',
 b'{"order_id": "1764751057914", "symbol": "NSE:BANKNIFTY25DEC59600CE", "price": 614.9, "qty": 100, "status": "SELL", "timestamp": 1764751057.9142118}',
 b'{"order_id": "1764751060891", "symbol": "NSE:BANKNIFTY25DEC59600CE", "price": 613.05, "qty": 100, "status": "BUY", "timestamp": 1764751060.8919673}',
 b'{"order_id": "1764751060996", "symbol": "NSE:BANKNIFTY25DEC59600CE", "price": 616.0, "qty": 100, "status": "SELL", "timestamp": 1764751060.9964478}',
 b'{"order_id": "1764751064516", "symbol": "NSE:BANKNIFTY25DEC59600CE", "price": 618

In [8]:
class FyersCostCalculator:
    """
    Calculates the exact cost of a trade on Fyers for NSE Options.
    Calibrated to match Fyers Portal (April 2024/2025 Revised Rates).
    """
    
    def __init__(self):
        # RATES (As fractions)
        self.BROKERAGE_PER_ORDER = 20.0
        
        # Sell Side Only (0.1%)
        self.STT_RATE = 0.001          
        
        # Exchange Txn Rate (NSE Options)
        # Calibrated from screenshot: 57.327 / 130200 = 0.0004403
        self.EXCHANGE_TXN_RATE = 0.0004403 
        
        # Investor Protection Fund Trust (NSE IPFT)
        # Calibrated from screenshot: 0.65 / 130200 = 0.000005
        self.IPFT_RATE = 0.000005
        
        # Stamp Duty (Buy Side Only, 0.003%)
        self.STAMP_DUTY_RATE = 0.00003 
        
        # SEBI Turnover Fee (0.0001%)
        self.SEBI_FEE_RATE = 0.000001  
        
        # GST (18%)
        self.GST_RATE = 0.18           

    def calculate_trade_cost(self, buy_price, sell_price, quantity):
        """
        Returns a dictionary with total tax and net PnL.
        """
        # 1. Turnover
        buy_turnover = buy_price * quantity
        sell_turnover = sell_price * quantity
        total_turnover = buy_turnover + sell_turnover

        # 2. Brokerage (Flat ₹20 per leg => ₹40 total)
        brokerage = 40.0 

        # 3. STT (Sell Side Only for Options)
        stt = sell_turnover * self.STT_RATE

        # 4. Exchange Transaction Charges (NSE)
        etc = total_turnover * self.EXCHANGE_TXN_RATE

        # 5. NSE IPFT (Investor Protection Fund)
        ipft = total_turnover * self.IPFT_RATE

        # 6. Stamp Duty (Buy Side Only)
        stamp_duty = buy_turnover * self.STAMP_DUTY_RATE

        # 7. SEBI Turnover Fees
        sebi_fees = total_turnover * self.SEBI_FEE_RATE

        # 8. GST (18%)
        # Important: Fyers calculates GST on (Brokerage + ETC + SEBI)
        # IPFT, STT, and Stamp Duty are usually GST-exempt or calculated differently
        taxable_amount = brokerage + etc + sebi_fees
        gst = taxable_amount * self.GST_RATE

        # TOTAL TAXES
        total_charges = brokerage + stt + etc + ipft + stamp_duty + sebi_fees + gst
        
        # PnL Calculation
        gross_pnl = (sell_price - buy_price) * quantity
        net_pnl = gross_pnl - total_charges
        
        return {
            "total_charges": round(total_charges, 2),
            "net_pnl": round(net_pnl, 2),
            "break_even_points": round(total_charges / quantity, 2)
        }

    def get_fast_breakeven(self, quantity):
        """
        HFT OPTIMIZATION:
        Returns an approximate number of points needed to break even.
        """
        # Based on calibration: 100 qty costs ~182 Rs.
        # Cost per quantity approx 1.83 Rs.
        # Fixed cost (Brokerage) is 40.
        # Variable cost is approx 0.11% of turnover.
        
        # Simplified for speed:
        # Cost ~= 40 (Fixed) + (Price * Qty * 0.0011)
        # But for simple approximation, we can use a safe buffer
        estimated_cost = 65.0 # For small lots (15-30)
        return estimated_cost / quantity

# Usage Example
if __name__ == "__main__":
    calc = FyersCostCalculator()
    
    # Scenario matching your screenshot
    # Buy: 650, Sell: 652, Qty: 100
    result = calc.calculate_trade_cost(buy_price=1000, sell_price=1002, quantity=100)
    
    print(f"Gross Profit: {(1010-1000)*100}")
    print(f"Total Charges: ₹{result['total_charges']}")
    print(f"Net PnL: ₹{result['net_pnl']}")
    print(f"Points needed to survive: {result['break_even_points']}")

Gross Profit: 1000
Total Charges: ₹255.65
Net PnL: ₹-55.65
Points needed to survive: 2.56


In [2]:
import numpy as np
class FyersEquityCalculator:
    """
    Calculates cost for Fyers Intraday Equity.
    Calibrated from your screenshot (image_0cae9d.png).
    Supports Vectorized Operations (Pandas/Numpy).
    """
    
    def __init__(self):
        # Brokerage: 0.03% or Rs 20 whichever is lower per order
        self.BROKERAGE_PCT = 0.0003
        self.BROKERAGE_CAP = 20.0
        
        # STT: 0.025% (Sell Side Only for Intraday Equity)
        self.STT_RATE = 0.00025          
        
        # Exchange Txn Rate (NSE Equity Intraday)
        # Calibrated: 3.457 / 116400 = 0.0000297
        self.EXCHANGE_TXN_RATE = 0.0000297
        
        # IPFT: 0.0001%
        # Calibrated: 0.12 / 116400 = 0.000001
        self.IPFT_RATE = 0.000001
        
        # Stamp Duty (Buy Side Only, 0.003%)
        self.STAMP_DUTY_RATE = 0.00003 
        
        # SEBI Turnover Fee (0.0001%)
        self.SEBI_FEE_RATE = 0.000001  
        
        # GST (18%)
        self.GST_RATE = 0.18           

    def calculate_trade_cost(self, buy_price, sell_price, quantity):
        # 1. Turnover
        buy_turnover = buy_price * quantity
        sell_turnover = sell_price * quantity
        total_turnover = buy_turnover + sell_turnover

        # 2. Brokerage (Variable logic for Equity)
        # USE NP.MINIMUM for Vectorized support (Pandas Series safe)
        
        # Buy Leg
        brok_buy = np.minimum(buy_turnover * self.BROKERAGE_PCT, self.BROKERAGE_CAP)
        # Sell Leg
        brok_sell = np.minimum(sell_turnover * self.BROKERAGE_PCT, self.BROKERAGE_CAP)
        
        brokerage = brok_buy + brok_sell

        # 3. STT (Sell Side Only)
        stt = sell_turnover * self.STT_RATE

        # 4. Exchange Transaction Charges
        etc = total_turnover * self.EXCHANGE_TXN_RATE

        # 5. IPFT
        ipft = total_turnover * self.IPFT_RATE

        # 6. Stamp Duty (Buy Side Only)
        stamp_duty = buy_turnover * self.STAMP_DUTY_RATE

        # 7. SEBI Fees
        sebi_fees = total_turnover * self.SEBI_FEE_RATE

        # 8. GST (On Brokerage + ETC + SEBI)
        # Note: IPFT is excluded from GST base in Equity intraday usually
        taxable_amount = brokerage + etc + sebi_fees
        gst = taxable_amount * self.GST_RATE

        # TOTAL
        total_charges = brokerage + stt + etc + ipft + stamp_duty + sebi_fees + gst
        
        gross_pnl = (sell_price - buy_price) * quantity
        net_pnl = gross_pnl - total_charges
        
        return {
            "total_charges": np.round(total_charges, 2),
            "net_pnl": np.round(net_pnl, 2),
            "break_even_points": np.round(total_charges / quantity, 2)
        }

if __name__ == '__main__':
    calc = FyersEquityCalculator()
    # Scenario matching your screenshot
    # Buy: 650, Sell: 652, Qty: 100
    result = calc.calculate_trade_cost(buy_price=650, sell_price=652, quantity=1000)
    
    print(f"Gross Profit: {(652-650)*1000}")
    print(f"Total Charges: ₹{result['total_charges']}")
    print(f"Net PnL: ₹{result['net_pnl']}")
    print(f"Points needed to survive: {result['break_even_points']}")

Gross Profit: 2000
Total Charges: ₹278.17
Net PnL: ₹1721.83
Points needed to survive: 0.28


In [118]:
import pandas as pd

df = pd.read_csv("executed_orders.csv")

# Sort by time to maintain correct order
df = df.sort_values("Timestamp").reset_index(drop=True)
calc = FyersCostCalculator()
# Group by symbol to avoid mixing PE / CE etc.
results = []

for symbol, group in df.groupby("Symbol"):
    buys = group[group["Status"] == "BUY"].reset_index(drop=True)
    sells = group[group["Status"] == "SELL"].reset_index(drop=True)

    # Pairing trades (round trips)
    pairs = min(len(buys), len(sells))
    buys = buys.head(pairs)
    sells = sells.head(pairs)

    pnl = (sells["Price"] - buys["Price"]) * 100
    pnl_after_charges = calc.calculate_trade_cost(buy_price=buys["Price"], sell_price=sells["Price"], quantity=100)

    temp = pd.DataFrame({
        "Symbol": symbol,
        "Buy_Time": buys["Timestamp"],
        "Buy_Price": buys["Price"],
        "Sell_Time": sells["Timestamp"],
        "Sell_Price": sells["Price"],
        "Quantity": buys["Quantity"],
        "PNL": pnl,
        "Total_Charges": pnl_after_charges["total_charges"],
        "Net_PNL": pnl_after_charges["net_pnl"]
    })

    results.append(temp)

# Combine all results
final_df = pd.concat(results, ignore_index=True)

# Save
final_df.to_csv("pnl_output.csv", index=False)

print(final_df.head())
print("\nTOTAL PNL =", final_df["PNL"].sum())


                      Symbol      Buy_Time  Buy_Price     Sell_Time  \
0  NSE:BANKNIFTY25DEC59600CE  1.764743e+09     637.75  1.764743e+09   
1  NSE:BANKNIFTY25DEC59600CE  1.764743e+09     633.45  1.764743e+09   
2  NSE:BANKNIFTY25DEC59600CE  1.764743e+09     630.90  1.764743e+09   
3  NSE:BANKNIFTY25DEC59600CE  1.764743e+09     632.55  1.764743e+09   
4  NSE:BANKNIFTY25DEC59600CE  1.764743e+09     628.20  1.764743e+09   

   Sell_Price  Quantity    PNL  Total_Charges  Net_PNL  
0      636.40       100 -135.0         179.74  -314.74  
1      635.50       100  205.0         179.36    25.64  
2      629.80       100 -110.0         178.35  -288.35  
3      634.55       100  200.0         179.17    20.83  
4      626.25       100 -195.0         177.66  -372.66  

TOTAL PNL = 12030.000000000075


In [91]:
import csv
import json

with open('executed_orders_2nd.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Order Id', 'Timestamp', 'Symbol', 'Quantity', 'Price', 'Status'])  # Header
    for record in data:
        order = json.loads(record.decode('utf-8'))  # decode bytes → str → dict
        writer.writerow([
            order['order_id'],
            order['timestamp'],
            order['symbol'],
            order['qty'],
            order['price'],
            order['status']
        ])


In [97]:
class FyersCostCalculator:
    """
    Calculates the exact cost of a trade on Fyers for NSE Options.
    Calibrated to match Fyers Portal (April 2024/2025 Revised Rates).
    """
    
    def __init__(self):
        # RATES (As fractions)
        self.BROKERAGE_PER_ORDER = 20.0
        
        # Sell Side Only (0.1%)
        self.STT_RATE = 0.001          
        
        # Exchange Txn Rate (NSE Options)
        # Calibrated from screenshot: 57.327 / 130200 = 0.0004403
        self.EXCHANGE_TXN_RATE = 0.0004403 
        
        # Investor Protection Fund Trust (NSE IPFT)
        # Calibrated from screenshot: 0.65 / 130200 = 0.000005
        self.IPFT_RATE = 0.000005
        
        # Stamp Duty (Buy Side Only, 0.003%)
        self.STAMP_DUTY_RATE = 0.00003 
        
        # SEBI Turnover Fee (0.0001%)
        self.SEBI_FEE_RATE = 0.000001  
        
        # GST (18%)
        self.GST_RATE = 0.18           

    def calculate_trade_cost(self, buy_price, sell_price, quantity):
        """
        Returns a dictionary with total tax and net PnL.
        """
        # 1. Turnover
        buy_turnover = buy_price * quantity
        sell_turnover = sell_price * quantity
        total_turnover = buy_turnover + sell_turnover

        # 2. Brokerage (Flat ₹20 per leg => ₹40 total)
        brokerage = 40.0 

        # 3. STT (Sell Side Only for Options)
        stt = sell_turnover * self.STT_RATE

        # 4. Exchange Transaction Charges (NSE)
        etc = total_turnover * self.EXCHANGE_TXN_RATE

        # 5. NSE IPFT (Investor Protection Fund)
        ipft = total_turnover * self.IPFT_RATE

        # 6. Stamp Duty (Buy Side Only)
        stamp_duty = buy_turnover * self.STAMP_DUTY_RATE

        # 7. SEBI Turnover Fees
        sebi_fees = total_turnover * self.SEBI_FEE_RATE

        # 8. GST (18%)
        # Important: Fyers calculates GST on (Brokerage + ETC + SEBI)
        # IPFT, STT, and Stamp Duty are usually GST-exempt or calculated differently
        taxable_amount = brokerage + etc + sebi_fees
        gst = taxable_amount * self.GST_RATE

        # TOTAL TAXES
        total_charges = brokerage + stt + etc + ipft + stamp_duty + sebi_fees + gst
        
        # PnL Calculation
        gross_pnl = (sell_price - buy_price) * quantity
        net_pnl = gross_pnl - total_charges
        
        return {
            "total_charges": round(total_charges, 2),
            "net_pnl": round(net_pnl, 2),
            "break_even_points": round(total_charges / quantity, 2)
        }

    def get_fast_breakeven(self, quantity):
        """
        HFT OPTIMIZATION:
        Returns an approximate number of points needed to break even.
        """
        # Based on calibration: 100 qty costs ~182 Rs.
        # Cost per quantity approx 1.83 Rs.
        # Fixed cost (Brokerage) is 40.
        # Variable cost is approx 0.11% of turnover.
        
        # Simplified for speed:
        # Cost ~= 40 (Fixed) + (Price * Qty * 0.0011)
        # But for simple approximation, we can use a safe buffer
        estimated_cost = 65.0 # For small lots (15-30)
        return estimated_cost / quantity

# Usage Example
if __name__ == "__main__":
    calc = FyersCostCalculator()
    
    # Scenario matching your screenshot
    # Buy: 650, Sell: 652, Qty: 100
    result = calc.calculate_trade_cost(buy_price=633.45, sell_price=635.5, quantity=100)
    
    print(f"Gross Profit: {(635.5-633.45)*100}")
    print(f"Total Charges: ₹{result['total_charges']}")
    print(f"Net PnL: ₹{result['net_pnl']}")
    print(f"Points needed to survive: {result['break_even_points']}")

Gross Profit: 204.99999999999545
Total Charges: ₹179.36
Net PnL: ₹25.64
Points needed to survive: 1.79


In [83]:
class FyersCostCalculator:
    """
    Calculates the exact cost of a trade on Fyers for NSE Options.
    Calibrated to match Fyers Portal (April 2024/2025 Revised Rates).
    """
    
    def __init__(self):
        # RATES (As fractions)
        self.BROKERAGE_PER_ORDER = 20.0
        
        # Sell Side Only (0.1%)
        self.STT_RATE = 0.001          
        
        # Exchange Txn Rate (NSE Options)
        # Calibrated from screenshot: 57.327 / 130200 = 0.0004403
        self.EXCHANGE_TXN_RATE = 0.0004403 
        
        # Investor Protection Fund Trust (NSE IPFT)
        # Calibrated from screenshot: 0.65 / 130200 = 0.000005
        self.IPFT_RATE = 0.000005
        
        # Stamp Duty (Buy Side Only, 0.003%)
        self.STAMP_DUTY_RATE = 0.00003 
        
        # SEBI Turnover Fee (0.0001%)
        self.SEBI_FEE_RATE = 0.000001  
        
        # GST (18%)
        self.GST_RATE = 0.18           

    def calculate_trade_cost(self, buy_price, sell_price, quantity):
        """
        Returns a dictionary with total tax and net PnL.
        """
        # 1. Turnover
        buy_turnover = buy_price * quantity
        sell_turnover = sell_price * quantity
        total_turnover = buy_turnover + sell_turnover

        # 2. Brokerage (Flat ₹20 per leg => ₹40 total)
        brokerage = 40.0 

        # 3. STT (Sell Side Only for Options)
        stt = sell_turnover * self.STT_RATE

        # 4. Exchange Transaction Charges (NSE)
        etc = total_turnover * self.EXCHANGE_TXN_RATE

        # 5. NSE IPFT (Investor Protection Fund)
        ipft = total_turnover * self.IPFT_RATE

        # 6. Stamp Duty (Buy Side Only)
        stamp_duty = buy_turnover * self.STAMP_DUTY_RATE

        # 7. SEBI Turnover Fees
        sebi_fees = total_turnover * self.SEBI_FEE_RATE

        # 8. GST (18%)
        # Important: Fyers calculates GST on (Brokerage + ETC + SEBI)
        # IPFT, STT, and Stamp Duty are usually GST-exempt or calculated differently
        taxable_amount = brokerage + etc + sebi_fees
        gst = taxable_amount * self.GST_RATE

        # TOTAL TAXES
        total_charges = brokerage + stt + etc + ipft + stamp_duty + sebi_fees + gst
        
        # PnL Calculation
        gross_pnl = (sell_price - buy_price) * quantity
        net_pnl = gross_pnl - total_charges
        
        return {
            "total_charges": round(total_charges, 2),
            "net_pnl": round(net_pnl, 2),
            "break_even_points": round(total_charges / quantity, 2)
        }

    def get_fast_breakeven(self, quantity):
        """
        HFT OPTIMIZATION:
        Returns an approximate number of points needed to break even.
        """
        # Based on calibration: 100 qty costs ~182 Rs.
        # Cost per quantity approx 1.83 Rs.
        # Fixed cost (Brokerage) is 40.
        # Variable cost is approx 0.11% of turnover.
        
        # Simplified for speed:
        # Cost ~= 40 (Fixed) + (Price * Qty * 0.0011)
        # But for simple approximation, we can use a safe buffer
        estimated_cost = 65.0 # For small lots (15-30)
        return estimated_cost / quantity

    def analyze_efficiency(self, buy_price, sell_price, quantity):
        """
        New Tool: Explains WHERE the money is going.
        """
        res = self.calculate_trade_cost(buy_price, sell_price, quantity)
        gross = (sell_price - buy_price) * quantity
        tax_pct = (res['total_charges'] / gross) * 100 if gross > 0 else 0
        
        print(f"--- 📊 TRADE EFFICIENCY REPORT ---")
        print(f"Scenario: Buy {buy_price} -> Sell {sell_price} (Qty: {quantity})")
        print(f"Gross Profit:  ₹{gross:.2f}")
        print(f"Total Taxes:   ₹{res['total_charges']:.2f}")
        print(f"Net Profit:    ₹{res['net_pnl']:.2f}")
        print(f"Tax Burn Rate: {tax_pct:.1f}% of your profit is gone in taxes.")
        
        if tax_pct > 50:
            print("\n⚠️  WARNING: SCALPER'S TRAP DETECTED")
            print("You are aiming for too few points on a high-value asset.")
            print("Regulatory charges (STT + Exchange) are eating your edge.")
            print(f"Recommendation: Minimum Target should be > {res['break_even_points'] * 2:.2f} points.")
        print("-----------------------------------")

# Usage Example
if __name__ == "__main__":
    calc = FyersCostCalculator()
    
    # Analyze the "Trap" scenario
    calc.analyze_efficiency(buy_price=650, sell_price=648, quantity=100)

--- 📊 TRADE EFFICIENCY REPORT ---
Scenario: Buy 650 -> Sell 648 (Qty: 100)
Gross Profit:  ₹-200.00
Total Taxes:   ₹182.19
Net Profit:    ₹-382.19
Tax Burn Rate: 0.0% of your profit is gone in taxes.
-----------------------------------
